In [1]:
import time
import sys
import tomoDataClass
from datetime import datetime
from helperFunctions import DualLogger, convert_to_tiff, subpixel_shift

# -------------------------
# CONFIGURATION FLAGS
# -------------------------
log = False # Set to True to enable logging output to a file
saveToFile = True     # Set to True to save aligned projection data to a TIFF file

# -------------------------
# SETUP: Timing & Logging
# -------------------------
start_time = time.time()  # Start execution timer
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")  # Generate timestamp for filenames
print("timestamp:", timestamp)

if log:
    # Redirect stdout/stderr to both console and log file
    sys.stdout = DualLogger(f'logs/output_tomoMono_align{timestamp}.txt', 'w')

timestamp: 20250917-235121


In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
filename = r"/Users/levihancock/Downloads/poly_tomo_128.hdf5"

indicesToRemove = [0,1,2,3,4,5,53,60,67,69,82,90,178,183,186, 209, 218, 219, 221, 224, 254, 258, 276, 278, 304, 306] + list(range(308,325)) ##Look out for 200, 

with h5py.File(filename, "r") as f:
    data = np.array(f["/data"])
    angles = list(f["/angles"])

In [3]:
#removing images with holder in the way
print("data shape is: ", data.shape)
print("angles shape is: ", len(angles))
# for index in reversed(indicesToRemove):
#     print(index)
data = np.delete(data, indicesToRemove, axis = 0)
angles = np.delete(angles, indicesToRemove, axis = 0)

data = data[::4]
angles = angles[::4]
print("data shape after removing is: ", data.shape)
print("angles shape after removing is: ", len(angles))

tomo = tomoDataClass.tomoData(data, angles)
scale_info = None

data shape is:  (325, 662, 2322)
angles shape is:  325
data shape after removing is:  (71, 662, 2322)
angles shape after removing is:  71


In [ ]:
print("Starting alignment")

savePath = f"alignedProjections/APSbeamtime1/aligned_XC_{timestamp}.tif"
print("\n\nCreating aligned projections:", savePath)

# Ensure alignment begins from original, unmodified projections
tomo.reset_workingProjections(x_size = data.shape[2], y_size=data.shape[1])

# -------------------------
# ALIGNMENT STRATEGY
# -------------------------
# Choose and configure alignment algorithm below:
tomo.shift_min_to_middle()
tomo.cross_correlate_align(tolerance=0.01, max_iterations = 1, yROI_Range=[50, -85], xROI_Range=[400, -400], maxShiftTolerance=5, isFull360=False)
tomo.PMA(max_iterations = 1, tolerance=0.0001, algorithm="sirt", crop_bottom_center_y = tomo.workingProjections.shape[1]-80, crop_bottom_center_x = 1200, isPhaseData = True)
tomo.center_projections()
tomo.make_updates_shift()
tomo.makeNotebookProjMovie()

Starting alignment


Creating aligned projections: alignedProjections/APSbeamtime1/aligned_XC_20250917-235121.tif


Shifting min values to middle


Shifting min to middle: 100%|███████████████████| 71/71 [00:24<00:00,  2.91it/s]




Cross-Correlation Alignment


Iteration 1/1: 100%|████████████████████████████| 70/70 [00:27<00:00,  2.53it/s]


Average pixel shift of iteration 1: 166.56734440606087
Maximum iterations reached without convergence.


Projection Matching Alignment (PMA)
Centering Projections
Original center: 656.0
Center of frame: 600
Aligned projections shifted by -56.0 pixels
Projections are currently centered at pixel 618.25. Residual offset: 18.25
Original center: 618.25
Center of frame: 600
Aligned projections shifted by -18.25 pixels
Projections are currently centered at pixel 600.0. Residual offset: 0.0


PMA Algorithm iterations:   0%|                           | 0/1 [00:00<?, ?it/s]

In [ ]:
print(len(tomo.ang))
print(tomo.workingProjections.shape)